<a href="https://colab.research.google.com/github/VarunB1234/Depression-Detection/blob/main/WithGating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import nltk
nltk.download('stopwords')
import numpy as np
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
import gc
import math
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from smart_open import open
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from keras.layers import Dropout
from matplotlib import pyplot as plt

dev_location = "dev_data"
test_location = "test_data"
train_location = "train_data"

devData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
testData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
trainData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]

dataset = np.concatenate((devData, np.concatenate((testData, trainData))))

gc.collect()
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/MCA Dataset/Copy of GoogleNews-vectors-negative300.bin', binary=True)
stop_words = set(stopwords.words('english'))

Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


ModuleNotFoundError: No module named 'gensim'

In [5]:
!pip install gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 66.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import nltk
nltk.download('stopwords')
import numpy as np
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
import gc
import math
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from smart_open import open
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from keras.layers import Dropout
from matplotlib import pyplot as plt

dev_location = "dev_data"
test_location = "test_data"
train_location = "train_data"

devData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
testData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
trainData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]

dataset = np.concatenate((devData, np.concatenate((testData, trainData))))

gc.collect()
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/MCA Dataset/GoogleNews-vectors-negative300.bin', binary=True)
stop_words = set(stopwords.words('english'))

Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
def checkDataPointExistence(patientID, split):
  for i in split:
    if(patientID == i[0]):
      return True
  return False

def getData(patientID, location):
  print("PatientID: " + str(int(patientID)))
  retData = [int(patientID)]
  textD = getTextData(patientID, location)
  textD = np.array(textD)
  audioD = getAudioData(patientID, location, textD)
  videoD = getVideoData(str(int(patientID)), location, textD)
  # patientD = np.concatenate((textD, audioD, videoD), axis = 1)
  # print("Final Patient Data: " + str(patientD.shape))
  # patientD = textD

  return textD,audioD,videoD

def getTextData(patientID, location):
  fileName = "/content/drive/My Drive/MCA Dataset/"+ str(location) + "/" + str(int(patientID)) + "_TRANSCRIPT.csv"
  file = np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8', engine='python'))
  outputList = []

  for i in range(len(file)):
    sentence = file[i][3]
    words = str(sentence).split(" ")
    totalWordLength = 0
    for word in words:
      totalWordLength += len(word)

    sentenceStart = file[i][0]
    sentenceEnd = file[i][1]
    speaker = file[i][2]
    if speaker == 'Ellie':
      continue
    else:
      speaker = 1

    totalTime = sentenceEnd - sentenceStart
    timePerChar = totalTime/totalWordLength

    wordStart = sentenceStart
    for word in words:
      wordEnd = wordStart + (timePerChar * len(word))
      appender = [wordStart, wordEnd, speaker]
      vector = list(returnWordToVec(word))
      for v in vector:
        appender.append(v)
      # print(appender)
      # print(len(appender))
      outputList.append(appender)
      wordStart = wordEnd

  return outputList

def remove_StopWords(sentence):
    filtered_sentence = []
    for w in sentence:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

def returnWordToVec(word):
  word = str(word)
  try:
    if word[0] == '<':
        word = word[1:]
    if word[-1] == '>':
        word = word[0:-1]
  except:
    dfdsd = 3
  if(word in model):
      return np.array(model[word])
  else:
    return np.zeros((300))

def audioDataHelper(X):
    for i in range(X.shape[0]):
        if(X[i,1] == 0):
            X[i,0] = 0
            for j in range(7):
                X[i,j+1] = 0
    X = np.array(X)
    return X

def getAudioData(patientID, location, textD):
  fileName = "/content/drive/My Drive/MCA Dataset/"+ str(location) + "/" + str(int(patientID)) + "_COVAREP.csv"
  data = pd.read_csv(fileName,header = None)
  data = data.iloc[:,:].values
  data = audioDataHelper(data)
  # print("Audio Raw Data:" + str(data.shape))
  sentenceDatas = []
  for sentence in textD:
    sentenceStartime = sentence[0]
    sentenceEndTime = sentence[1]
    startIndex = math.floor(sentenceStartime/0.01)
    endIndex = math.ceil(sentenceEndTime/0.01)
    sentenceData = data[startIndex: endIndex]
    sentenceData = np.average(sentenceData, axis = 0)
    sentenceData = np.array(sentenceData.reshape(1, -1))
    sentenceDatas.append(sentenceData)
  sentenceDatas = np.array(sentenceDatas)
  sentenceDatas = np.reshape(sentenceDatas, (textD.shape[0],-1))

  return sentenceDatas

def getVideoDataHelper(patientID, location):
  root = "/content/drive/My Drive/MCA Dataset/"+ str(location) + "/"
  file1 = root + (patientID)+"_CLNF_AUs.txt"
  file2 = root + (patientID)+"_CLNF_features.txt"
  file3 = root + (patientID)+"_CLNF_features3D.txt"
  file4 = root + (patientID)+"_CLNF_gaze.txt"
  file6 = root + (patientID)+"_CLNF_pose.txt"
  data = processVideoData(file1)
  data = np.concatenate((data, processVideoData(file2)), 1)
  data = np.concatenate((data, processVideoData(file3)), 1)
  data = np.concatenate((data, processVideoData(file4)), 1)
  data = np.concatenate((data, processVideoData(file6)), 1)

  # try:
  #   hogFile = open(file5, 'r')
  # except:
  #   print("Hog tXT file")
  #   hogFile = open(root + (patientID)+"_CLNF_hog.txt", 'r')
  # hogData = np.fromfile(hogFile, dtype = np.uint32)
  # hogData = np.reshape(hogData, (-1, 4468))

  # data = np.concatenate((data, hogData), 1)
  return data

def processVideoData(filename):
  try:
    data = pd.read_csv(filename,delimiter=',', dtype=float)
    X = data.iloc[:,:].values
    X = np.delete(X, 0, 1)
    X = np.delete(X, 1, 1)
  except:
    print("Video Data corrupt, fixing.")
    data = pd.read_csv(filename,delimiter=',')
    X = data.iloc[:,:].values
    X = np.delete(X, 0, 1)
    X = np.delete(X, 1, 1)
    for i in range(len(X)):
        if(isinstance(X[i][5],str) or isinstance(X[i][7],str)):
            X[i] = np.zeros((1, X.shape[1]))
  return X

def getVideoData(patientID, location, textD):
  data = getVideoDataHelper(patientID, location)
  sentenceDatas = []
  for sentence in textD:
    sentenceStartime = sentence[0]
    sentenceEndTime = sentence[1]
    startIndex = math.floor(sentenceStartime/0.333)
    endIndex = math.ceil(sentenceEndTime/0.333)
    sentenceData = data[startIndex: endIndex]
    sentenceData = np.average(sentenceData, axis = 0)
    sentenceData = np.array(sentenceData.reshape(1, -1))
    sentenceDatas.append(sentenceData)

  sentenceDatas = np.array(sentenceDatas)
  sentenceDatas = np.reshape(sentenceDatas, (textD.shape[0],-1))
  return sentenceDatas

# Xtrain = []
Ytrain = []
# Xtest = []
Ytest = []

audio_train = []
video_train = []
text_train = []
audio_test = []
video_test = []
text_test = []


lengths = []
for datapoint in dataset:
  # print(datapoint[0])
  if(checkDataPointExistence(datapoint[0], devData)):

   # Data Point in DevSet
    text,audio,video = getData(datapoint[0], dev_location)
    text = text[:,2:]
    audio_train.append(audio)
    video_train.append(video)
    text_train.append(text)
    Ytrain.append(datapoint[1])
    # print(data)
  if(checkDataPointExistence(datapoint[0], testData)):
    # Data Point in Test Set
    text,audio,video = getData(datapoint[0], test_location)
    text = text[:,2:]
    audio_test.append(audio)
    video_test.append(video)
    text_test.append(text)
    Ytest.append(datapoint[1])
  elif(checkDataPointExistence(datapoint[0], trainData)):
    # Data Point in Train Set
    text,audio,video = getData(datapoint[0], train_location)
    text = text[:,2:]
    audio_train.append(audio)
    video_train.append(video)
    text_train.append(text)
    Ytrain.append(datapoint[1])
  # lengths.append(data.shape[0])
model = []
gc.collect()


def refactor(arr, size):
  temp = arr[0:min(len(arr),size), :]
  if (len(temp) < size):
    temp = np.concatenate((temp, np.zeros(((size - len(temp)), arr.shape[1]))), axis = 0 )
  return temp

numberOfSentences = 1700

for i in range(len(audio_train)):
  audio_train[i] = refactor(audio_train[i], numberOfSentences)
  video_train[i] = refactor(video_train[i], numberOfSentences)
  text_train[i] = refactor(text_train[i], numberOfSentences)

for i in range(len(audio_test)):
  audio_test[i] = refactor(audio_train[i], numberOfSentences)
  video_test[i] = refactor(video_train[i], numberOfSentences)
  text_test[i] = refactor(text_train[i], numberOfSentences)

audio_test = np.array(audio_test)
video_test = np.array(video_test)
text_test = np.array(text_test)

audio_train = np.array(audio_train)
video_train = np.array(video_train)
text_train = np.array(text_train)

gc.collect()

print(audio_test.shape,video_test.shape,text_test.shape)
print(audio_train.shape,video_train.shape,text_train.shape)

dataset = []
devData = []
trainData = []
testdata = []
gc.collect()

Ytrain = np.array(Ytrain)
Ytest = np.array(Ytest)

gc.collect()

import sklearn
from sklearn import preprocessing

def upsample(X_train,Y_train):
  X_train_0 = X_train[Y_train==0]
  X_train_1 = X_train[Y_train==1]

  Y_train_1 = Y_train[Y_train==1]
  size = X_train_0.shape[0] - X_train_1.shape[0]
  X = []
  Y = []
  X_train = list(X_train)
  Y_train = list(Y_train)
  while(size>0):
    size -= 1
    index = np.random.randint(0,X_train_1.shape[0]-1)
    leave_index = np.random.randint(0,len(X_train)-1)
    X_add = X_train_1[index]
    X_leave = X_train[leave_index]

    Y_add = Y_train_1[index]
    Y_leave = Y_train[leave_index]

    X_train[leave_index] = X_add
    X_train.append(X_leave)

    Y_train[leave_index] = Y_add
    Y_train.append(Y_leave)


  X_train = np.array(X_train)
  Y_train = np.array(Y_train)
  return X_train,Y_train


audio_train = np.nan_to_num(audio_train)
video_train = np.nan_to_num(video_train)
text_train = np.nan_to_num(text_train)


audio_train, _ = upsample(audio_train,Ytrain)
video_train, _ = upsample(video_train,Ytrain)
text_train, Ytrain = upsample(text_train,Ytrain)

gc.collect()

for i in range(audio_train.shape[0]):
  audio_train[i] = sklearn.preprocessing.normalize(audio_train[i])
  video_train[i] = sklearn.preprocessing.normalize(video_train[i])
  text_train[i] = sklearn.preprocessing.normalize(text_train[i])


audio_test = np.nan_to_num(audio_test)
video_test = np.nan_to_num(video_test)
text_test = np.nan_to_num(text_test)


for i in range(audio_test.shape[0]):
  audio_test[i] = sklearn.preprocessing.normalize(audio_test[i])
  video_test[i] = sklearn.preprocessing.normalize(video_test[i])
  text_test[i] = sklearn.preprocessing.normalize(text_test[i])

PatientID: 302


/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


PatientID: 307


/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


PatientID: 331


...

/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


PatientID: 457


/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


PatientID: 459
PatientID: 463
PatientID: 464
PatientID: 468
PatientID: 471
PatientID: 473
PatientID: 474
PatientID: 475
PatientID: 478
PatientID: 479
PatientID: 485
PatientID: 486
PatientID: 487
PatientID: 488
PatientID: 491
(47, 1700, 74) (47, 1700, 388) (47, 1700, 301)
(142, 1700, 74) (142, 1700, 388) (142, 1700, 301)


ValueError: Input contains NaN.

In [16]:
# Word Level - Text + Audio + Video

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

import numpy as np
from sklearn.metrics import classification_report

# -------- Highway Layer Definition --------
class Highway(layers.Layer):
    def __init__(self):
        super(Highway, self).__init__()

    def build(self, input_shape):
        n_sentences = input_shape[1]
        n_features = input_shape[2]

        carry_bias = keras.initializers.Constant(value=-1.0)
        carry_bias_2 = keras.initializers.Constant(value=0.1)
        random_dist = keras.initializers.RandomNormal(mean=0.0, stddev=0.1)

        self.W_T = self.add_weight(shape=(n_features, n_features), initializer=random_dist, trainable=True)
        self.b_T = self.add_weight(shape=(n_sentences, n_features), initializer=carry_bias, trainable=True)

        self.W = self.add_weight(shape=(n_features, n_features), initializer=random_dist, trainable=True)
        self.b = self.add_weight(shape=(n_sentences, n_features), initializer=carry_bias_2, trainable=True)

    def call(self, inputs):
        x = inputs
        T = tf.sigmoid(tf.matmul(x, self.W_T) + self.b_T)
        H = tf.nn.relu(tf.matmul(x, self.W) + self.b)
        C = 1.0 - T
        return H * T + x * C

# -------- Model Definition --------
# Audio Input
input1 = Input(shape=(1700, 74), name='Audio_input')
highway1 = Highway()(input1)
highway5 = Highway()(highway1)
highway6 = Highway()(highway5)
dense1 = Dense(74)(highway6)

# Video Input
input2 = Input(shape=(1700, 388), name='Video_input')
highway2 = Highway()(input2)
highway3 = Highway()(highway2)
highway4 = Highway()(highway3)
dense7 = Dense(200)(highway4)
dense2 = Dense(74)(dense7)

# Text Input
input3 = Input(shape=(1700, 301), name='Text_input')
dense6 = Dense(150)(input3)
dense3 = Dense(74)(dense6)

# Merge and LSTM
merge = concatenate([dense1, dense2, dense3], axis=1)
lstm = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(merge)
output = Dense(1, activation='sigmoid')(lstm)

model = Model(inputs=[input1, input2, input3], outputs=output)

print(model.summary())
plot_model(model)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy')

# -------- Preprocess Input Data --------
# Ensure everything is a numpy array with correct dtype
audio_train = np.array(audio_train, dtype=np.float32)
video_train = np.array(video_train, dtype=np.float32)
text_train  = np.array(text_train, dtype=np.float32)
Ytrain      = np.array(Ytrain, dtype=np.float32)

audio_test = np.array(audio_test, dtype=np.float32)
video_test = np.array(video_test, dtype=np.float32)
text_test  = np.array(text_test, dtype=np.float32)
Ytest      = np.array(Ytest, dtype=np.float32)

# -------- Model Training --------
model.fit(
    [audio_train, video_train, text_train], Ytrain,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True
        )
    ],
    epochs=10,
    batch_size=2
)

# -------- Thresholding + Evaluation --------
def Thresholding(Y_pred, threshold):
    return np.where(Y_pred < threshold, 0, 1)

pred = model.predict([audio_test, video_test, text_test])
pred2 = model.predict([audio_train, video_train, text_train])

print("\n--- Test Accuracy (Threshold 0.5) ---")
print(classification_report(Ytest, Thresholding(pred, 0.5)))

print("\n--- Train Accuracy ---")
print(classification_report(Ytrain, Thresholding(pred2, 0.5)))

# Optional Threshold Tuning
for t in [0.6, 0.4, 0.3, 0.7]:
    print(f"\n--- Threshold: {t} ---")
    print(classification_report(Ytest, Thresholding(pred, t)))


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Video_input         │ (None, 1700, 388) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Audio_input         │ (None, 1700, 74)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ highway_21          │ (None, 1700, 388) │  1,620,288 │ Video_input[0][0] │
│ (Highway)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ highway_18          │ (None, 1700, 74)  │    262,552 │ Audio_input[0][0] │
│ (Highway)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ highway_22          │ (None, 1700, 388) │  1,620,288 │ highway_21[0][0]  │
│ (Highway)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ highway_19          │ (None, 1700, 74)  │    262,552 │ highway_18[0][0]  │
│ (Highway)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ highway_23          │ (None, 1700, 388) │  1,620,288 │ highway_22[0][0]  │
│ (Highway)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Text_input          │ (None, 1700, 301) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ highway_20          │ (None, 1700, 74)  │    262,552 │ highway_19[0][0]  │
│ (Highway)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_19 (Dense)    │ (None, 1700, 200) │     77,800 │ highway_23[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_21 (Dense)    │ (None, 1700, 150) │     45,300 │ Text_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 1700, 74)  │      5,550 │ highway_20[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_20 (Dense)    │ (None, 1700, 74)  │     14,874 │ dense_19[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_22 (Dense)    │ (None, 1700, 74)  │     11,174 │ dense_21[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 5100, 74)  │          0 │ dense_18[0][0],   │
│ (Concatenate)       │                   │            │ dense_20[0][0],   │
│                     │                   │            │ dense_22[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 128)       │    103,936 │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_23 (Dense)    │ (None, 1)         │        129 │ lstm_3[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,907,283 (22.53 MB)

 Trainable params: 5,907,283 (22.53 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 437s 5s/step - loss: nan - val_loss: nan
Epoch 2/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 429s 5s/step - loss: nan - val_loss: nan
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 425s 5s/step - loss: nan - val_loss: nan
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 430s 5s/step - loss: nan - val_loss: nan
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 425s 5s/step - loss: nan - val_loss: nan
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 425s 5s/step - loss: nan - val_loss: nan
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 424s 5s/step - loss: nan - val_loss: nan
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 428s 5s/step - loss: nan - val_loss: nan
Epoch 9/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 424s 5s/step - loss: nan - val_loss: nan
Epoch 10/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 425s 5s/step - loss: nan - val_loss: nan
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step

--- Test Accuracy (Threshold 0.5) ---
              precision    recall  f1-score   support

         0.0       0.00      0.00    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m